In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

##Import Data

In [2]:
!curl -L -o ./20k-multi-class-crop-disease-images.zip\
  https://www.kaggle.com/api/v1/datasets/download/jawadali1045/20k-multi-class-crop-disease-images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2395M  100 2395M    0     0  39.8M      0  0:01:00  0:01:00 --:--:-- 40.9M


In [3]:
!unzip 20k-multi-class-crop-disease-images.zip


Streaming output truncated to the last 5000 lines.
  inflating: Train/Wilt/fus263.jpg   
  inflating: Train/Wilt/fus264.jpg   
  inflating: Train/Wilt/fus265.jpg   
  inflating: Train/Wilt/fus266.jpg   
  inflating: Train/Wilt/fus267.jpg   
  inflating: Train/Wilt/fus268.jpg   
  inflating: Train/Wilt/fus269.jpg   
  inflating: Train/Wilt/fus27.jpg    
  inflating: Train/Wilt/fus270.jpg   
  inflating: Train/Wilt/fus271.jpg   
  inflating: Train/Wilt/fus272.jpg   
  inflating: Train/Wilt/fus273.jpg   
  inflating: Train/Wilt/fus274.jpg   
  inflating: Train/Wilt/fus275.jpg   
  inflating: Train/Wilt/fus276.jpg   
  inflating: Train/Wilt/fus277.jpg   
  inflating: Train/Wilt/fus278.jpg   
  inflating: Train/Wilt/fus279.jpg   
  inflating: Train/Wilt/fus28.jpg    
  inflating: Train/Wilt/fus280.jpg   
  inflating: Train/Wilt/fus281.jpg   
  inflating: Train/Wilt/fus282.jpg   
  inflating: Train/Wilt/fus283.jpg   
  inflating: Train/Wilt/fus284.jpg   
  inflating: Train/Wilt/fus285.jpg   

In [4]:
data_dir = './Train'
image_size = (224,224)
batch_size = 32

##Rescaling, Rotation, Shifting & Zooming, Flipping& Shearing


In [5]:
train_data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [6]:
train_generator = train_data.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 12374 images belonging to 42 classes.


In [7]:
validation_generator = train_data.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3073 images belonging to 42 classes.


##Base Model VGG16

In [8]:
base_model = VGG16( weights= 'imagenet', include_top = False, input_shape=(224,224,3))
base_model.trainable = False
num_classes = train_generator.num_classes
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
epochs = 15
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
372/387 ━━━━━━━━━━━━━━━━━━━━ 8s 583ms/step - accuracy: 0.3416 - loss: 2.6022

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


387/387 ━━━━━━━━━━━━━━━━━━━━ 288s 709ms/step - accuracy: 0.3460 - loss: 2.5841 - val_accuracy: 0.6209 - val_loss: 1.4993
Epoch 2/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 261s 674ms/step - accuracy: 0.5790 - loss: 1.6353 - val_accuracy: 0.6515 - val_loss: 1.2894
Epoch 3/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 262s 678ms/step - accuracy: 0.6210 - loss: 1.3774 - val_accuracy: 0.6931 - val_loss: 1.1349
Epoch 4/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 259s 668ms/step - accuracy: 0.6672 - loss: 1.2317 - val_accuracy: 0.7019 - val_loss: 1.0398
Epoch 5/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 261s 675ms/step - accuracy: 0.6854 - loss: 1.1389 - val_accuracy: 0.7227 - val_loss: 0.9860
Epoch 6/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 296s 766ms/step - accuracy: 0.7128 - loss: 1.0426 - val_accuracy: 0.7423 - val_loss: 0.9126
Epoch 7/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 261s 673ms/step - accuracy: 0.7182 - loss: 0.9752 - val_accuracy: 0.7501 - val_loss: 0.8823
Epoch 8/15
387/387 ━━━━━━━━━━━━━━━━━━━━ 260s 672ms/step - accuracy: 0.7276 - loss: 0.95

In [11]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction) * 100  # Convert to percentage

    class_labels = list(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]


In [12]:
CropDiseaseSolution = {
    "American Bollworm on Cotton": {
        "Cause": "Larvae of Helicoverpa armigera feeding on cotton bolls.",
        "Peak Season": "Summer and early monsoon.",
        "Remedy": "Use pheromone traps and insecticides like Spinosad or Bacillus thuringiensis."
    },
    "Anthracnose on Cotton": {
        "Cause": "Fungal infection caused by Colletotrichum species.",
        "Peak Season": "High humidity periods, usually post-monsoon.",
        "Remedy": "Apply copper-based fungicides and ensure good field drainage."
    },
    "Army worm": {
        "Cause": "Larvae of Spodoptera species attacking foliage.",
        "Peak Season": "Rainy season and post-monsoon.",
        "Remedy": "Use neem oil or Bacillus thuringiensis-based biopesticides."
    },
    "Bacterial Blight in cotton": {
        "Cause": "Xanthomonas citri bacteria spreading through infected seeds and water.",
        "Peak Season": "Warm and humid conditions.",
        "Remedy": "Use resistant varieties and copper-based fungicides."
    },
    "Becterial Blight in Rice": {
        "Cause": "Xanthomonas oryzae pv. oryzae bacteria spread via rain and irrigation water.",
        "Peak Season": "Rainy season and high humidity.",
        "Remedy": "Avoid overhead irrigation and use copper oxychloride fungicides."
    },
    "Brownspot": {
        "Cause": "Drechslera oryzae fungus causing lesions on leaves.",
        "Peak Season": "High humidity and excessive nitrogen fertilization.",
        "Remedy": "Use balanced fertilization and Mancozeb fungicide spray."
    },
    "Common_Rust": {
        "Cause": "Puccinia sorghi fungus spreading through wind-borne spores.",
        "Peak Season": "Warm, humid conditions during late summer.",
        "Remedy": "Plant rust-resistant varieties and use sulfur-based fungicides."
    },
    "Cotton Aphid": {
        "Cause": "Aphis gossypii insects sucking sap from cotton plants.",
        "Peak Season": "Early growth stages in warm and dry conditions.",
        "Remedy": "Introduce natural predators like ladybugs and use neem oil spray."
    },
    "Flag Smut": {
        "Cause": "Urocystis agropyri fungus affecting wheat seedlings.",
        "Peak Season": "Cool and moist conditions during early growth.",
        "Remedy": "Use disease-free seeds and treat seeds with fungicides before planting."
    },
    "Gray_Leaf_Spot": {
        "Cause": "Fungal infection caused by Cercospora species.",
        "Peak Season": "Late summer and early autumn.",
        "Remedy": "Use strobilurin-based fungicides and improve air circulation in the field."
    },
    "Healthy Maize": {
        "Cause": "No disease detected.",
        "Peak Season": "N/A",
        "Remedy": "Crop is healthy, no diagnosis required."
    },
    "Healthy Wheat": {
        "Cause": "No disease detected.",
        "Peak Season": "N/A",
        "Remedy": "Crop is healthy, no diagnosis required."
    },
     "Healthy cotton": {
        "Cause": "No disease detected.",
        "Peak Season": "N/A",
        "Remedy": "Crop is healthy, no diagnosis required."
    },
    "Leaf Curl": {
        "Cause": "Begomovirus transmitted by whiteflies.",
        "Peak Season": "Monsoon and early winter.",
        "Remedy": "Control whiteflies, as they spread the virus, and use resistant plant varieties."
    },
    "Leaf smut": {
        "Cause": "Fungal infection caused by Entyloma oryzae.",
        "Peak Season": "Humid conditions in early growth stages.",
        "Remedy": "Use seed treatment with Thiram or Captan before sowing."
    },
    "Mosaic sugarcane": {
        "Cause": "Sugarcane mosaic virus (SCMV) spread by aphids.",
        "Peak Season": "Monsoon and post-monsoon.",
        "Remedy": "Use virus-free planting material and remove infected plants."
    },
    "RedRot sugarcane": {
        "Cause": "Fungal disease caused by Colletotrichum falcatum.",
        "Peak Season": "Warm and humid conditions.",
        "Remedy": "Apply Trichoderma viride-based biofungicides and remove infected stalks."
    },
    "Wheat Black Rust": {
        "Cause": "Fungal infection caused by Puccinia graminis.",
        "Peak Season": "Late winter and early spring.",
        "Remedy": "Apply triazole fungicides and use rust-resistant wheat varieties."
    },
    "Wilt": {
        "Cause": "Fungal infection caused by Fusarium oxysporum.",
        "Peak Season": "Hot and dry conditions.",
        "Remedy": "Use disease-free seeds and practice crop rotation."
    },
    "Yellow Rust Sugarcane": {
        "Cause": "Fungal infection caused by Puccinia kuehnii.",
        "Peak Season": "Cool and humid conditions.",
        "Remedy": "Use resistant varieties and sulfur fungicides."
    },
    "maize ear rot": {
        "Cause": "Fungal infection caused by Fusarium species.",
        "Peak Season": "High moisture conditions during grain filling.",
        "Remedy": "Harvest early and ensure proper drying of maize cobs."
    },
    "maize fall armyworm": {
        "Cause": "Spodoptera frugiperda larvae feeding on maize leaves.",
        "Peak Season": "Warm and humid conditions.",
        "Remedy": "Use biological control like parasitoid wasps and neem oil sprays."
    },
    "pink bollworm in cotton": {
        "Cause": "Pectinophora gossypiella larvae burrowing into cotton bolls.",
        "Peak Season": "Warm and dry conditions during boll formation.",
        "Remedy": "Use pheromone traps and Bt cotton varieties."
    },
    "red cotton bug": {
        "Cause": "Dysdercus cingulatus sucking sap from cotton plants.",
        "Peak Season": "Post-monsoon and dry weather.",
        "Remedy": "Use insecticides like Malathion and remove plant debris after harvest."
    }
}

In [13]:
def get_disease_solution(img_path):
    predicted_disease, confidence = predict_image(img_path)  # Get disease prediction

    # Fetch solution from dictionary
    solution = CropDiseaseSolution.get(predicted_disease, None)

    print(f"\nPredicted Disease: {predicted_disease} ({confidence:.2f}%)")

    if solution:
        print(f"Cause: {solution['Cause']}")
        print(f"Peak Season: {solution['Peak Season']}")
        print(f"Remedy: {solution['Remedy']}")
    else:
        print("No specific solution available.")

In [15]:
model.save_weights('model.weights.h5')